In [19]:
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"

from tensorflow.python.client import device_lib
print (device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17671537360288316268
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 170459136
locality {
  bus_id: 1
}
incarnation: 9493394957667786037
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:65:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 338231296
locality {
  bus_id: 1
}
incarnation: 13832129856767949179
physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:b3:00.0, compute capability: 6.1"
]


In [20]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility


import gzip
import sys
if (sys.version_info > (3, 0)):
    import pickle as pkl
else: #Python 2.7 imports
    
    import cPickle as pkl

import keras
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten, concatenate
from keras.layers import Embedding
from keras.layers import Convolution1D, MaxPooling1D, GlobalMaxPooling1D
from keras.regularizers import Regularizer
from keras.preprocessing import sequence
import matplotlib.pyplot as plt

In [21]:
def wordIdxLookup(word, word_idx_map):
    if word in word_idx_map:
        return word_idx_map[word]

In [22]:
data = pkl.load(gzip.open("/home/dl1/Arav/Sentence_classic/code/pkl/data.pkl.gz","rb"))
print("data loaded!")

data loaded!


In [23]:
train_labels = data['train']['labels']
train_sentences = data['train']['sentences']

dev_labels = data['dev']['labels']
dev_sentences = data['dev']['sentences']

test_labels = data['test']['labels']
test_sentences = data['test']['sentences']

word_embeddings = data['wordEmbeddings']

In [24]:
# :: Find the longest sentence in our dataset ::
max_sentence_len = 0
for sentence in train_sentences + dev_sentences + test_sentences:
    max_sentence_len = max(len(sentence), max_sentence_len)

print("Longest sentence: %d" % max_sentence_len)

Longest sentence: 59


In [25]:
y_train = np.array(train_labels)
y_dev = np.array(dev_labels)
y_test = np.array(test_labels)

X_train = sequence.pad_sequences(train_sentences, maxlen=max_sentence_len)
X_dev = sequence.pad_sequences(dev_sentences, maxlen=max_sentence_len)
X_test = sequence.pad_sequences(test_sentences, maxlen=max_sentence_len)


print('X_train shape:', X_train.shape)
print('X_dev shape:', X_dev.shape)
print('X_test shape:', X_test.shape)

X_train shape: (5330, 59)
X_dev shape: (2664, 59)
X_test shape: (2668, 59)


In [26]:
y_train

array([0, 0, 0, ..., 1, 1, 1])

In [27]:
#  :: Create the network :: 

print('Build model...')

# set parameters:
batch_size = 50

nb_filter = 50
filter_lengths = [1,2,3]
hidden_dims = 100
nb_epoch = 100



words_input = Input(shape=(max_sentence_len,), dtype='int32', name='words_input')

Build model...


In [28]:
[words_input]
# word_embeddings.shape[1]

[<tf.Tensor 'words_input_1:0' shape=(?, 59) dtype=int32>]

In [29]:
#Our word embedding layer
wordsEmbeddingLayer = Embedding(word_embeddings.shape[0],
                    word_embeddings.shape[1],                                     
                    weights=[word_embeddings],
                    trainable=False)

words = wordsEmbeddingLayer(words_input)


In [30]:
# wordsEmbeddingLayer
words

<tf.Tensor 'embedding_2/Gather:0' shape=(?, 59, 300) dtype=float32>

In [31]:
#Now we add a variable number of convolutions
words_convolutions = []
for filter_length in filter_lengths:
    words_conv = Convolution1D(filters=nb_filter,
                            kernel_size=filter_length,
                            padding='same',
                            activation='relu',
                            strides=1)(words)
                            
    words_conv = GlobalMaxPooling1D()(words_conv)      
    
    words_convolutions.append(words_conv)  

output = concatenate(words_convolutions)



# We add a vanilla hidden layer together with dropout layers:
output = Dropout(0.5)(output)
output = Dense(hidden_dims, activation='tanh', kernel_regularizer=keras.regularizers.l2(0.01))(output)
output = Dropout(0.25)(output)


# We project onto a single unit output layer, and squash it with a sigmoid:
output = Dense(1, activation='sigmoid',  kernel_regularizer=keras.regularizers.l2(0.01))(output)

model = Model(inputs=[words_input], outputs=[output])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
words_input (InputLayer)        (None, 59)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 59, 300)      4966200     words_input[0][0]                
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 59, 50)       15050       embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 59, 50)       30050       embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_6 (

In [32]:
# for epoch in range(nb_epoch):
# print("\n------------- Epoch %d ------------" % (epoch+1))

trModel = model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch,
          verbose=1,validation_data=(X_dev,y_dev))


Train on 5330 samples, validate on 2664 samples
Epoch 1/100
5330/5330 [==============================] - 3s 500us/step - loss: 1.5313 - acc: 0.5739 - val_loss: 1.1277 - val_acc: 0.6989
Epoch 2/100
5330/5330 [==============================] - 1s 197us/step - loss: 0.9472 - acc: 0.7158 - val_loss: 0.7725 - val_acc: 0.7691
Epoch 3/100
5330/5330 [==============================] - 1s 198us/step - loss: 0.6715 - acc: 0.7829 - val_loss: 0.6215 - val_acc: 0.7785
Epoch 4/100
5330/5330 [==============================] - 1s 195us/step - loss: 0.5460 - acc: 0.8084 - val_loss: 0.5558 - val_acc: 0.7887
Epoch 5/100
5330/5330 [==============================] - 1s 195us/step - loss: 0.4638 - acc: 0.8358 - val_loss: 0.5462 - val_acc: 0.7744
Epoch 6/100
5330/5330 [==============================] - 1s 196us/step - loss: 0.4120 - acc: 0.8508 - val_loss: 0.5196 - val_acc: 0.7905
Epoch 7/100
5330/5330 [==============================] - 1s 197us/step - loss: 0.3713 - acc: 0.8775 - val_loss: 0.5193 - val_acc: 

5330/5330 [==============================] - 1s 197us/step - loss: 0.1122 - acc: 0.9904 - val_loss: 0.6223 - val_acc: 0.7932
Epoch 61/100
5330/5330 [==============================] - 1s 195us/step - loss: 0.1118 - acc: 0.9919 - val_loss: 0.6167 - val_acc: 0.7890
Epoch 62/100
5330/5330 [==============================] - 2s 447us/step - loss: 0.1090 - acc: 0.9921 - val_loss: 0.6409 - val_acc: 0.7842
Epoch 63/100
5330/5330 [==============================] - 1s 196us/step - loss: 0.1081 - acc: 0.9929 - val_loss: 0.6092 - val_acc: 0.7920
Epoch 64/100
5330/5330 [==============================] - 1s 195us/step - loss: 0.1060 - acc: 0.9934 - val_loss: 0.6413 - val_acc: 0.7827
Epoch 65/100
5330/5330 [==============================] - 1s 197us/step - loss: 0.1085 - acc: 0.9919 - val_loss: 0.6692 - val_acc: 0.7770
Epoch 66/100
5330/5330 [==============================] - 1s 195us/step - loss: 0.1067 - acc: 0.9942 - val_loss: 0.5927 - val_acc: 0.7864
Epoch 67/100
5330/5330 [=======================

In [33]:
#Use Keras to compute the loss and the accuracy
dev_loss, dev_accuracy = model.evaluate(X_dev, y_dev, verbose=1)
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)


print("Val-Accuracy: %.2f%% (loss: %.4f)" % (dev_accuracy*100, dev_loss))
print("Test-Accuracy: %.2f%% (loss: %.4f)" % (test_accuracy*100, test_loss))

2668/2668 [==============================] - 0s 84us/step
Val-Accuracy: 78.94% (loss: 0.6192)
Test-Accuracy: 78.15% (loss: 0.6318)


In [34]:
#Training Loss and Validation Loss
plt.figure(figsize=[8,6])
plt.plot(trModel.history['loss'],'r',linewidth=3.0)
plt.plot(trModel.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)

Text(0.5,1,'Loss Curves')

In [36]:
#Training Accuracy and Validation Accuracy
plt.figure(figsize=[8,6])
plt.plot(trModel.history['acc'],'r',linewidth=3.0)
plt.plot(trModel.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

Text(0.5,1,'Accuracy Curves')

In [37]:
preds = model.predict(X_test, batch_size=batch_size)

In [38]:
preds[0]

array([0.00644799], dtype=float32)

In [39]:
y_test[0]

0

In [40]:
model.save("/home/dl1/Arav/Sentence_classic/models/model1.h5")

In [41]:
from keras.models import load_model

loadedModel = load_model("/home/dl1/Arav/Sentence_classic/models/model1.h5")

In [54]:
result_data = pkl.load(gzip.open("/home/dl1/Arav/Sentence_classic/code/pkl/resultdata.pkl.gz","rb"))
print("data loaded!")

data loaded!


In [55]:
result_data.tolist()

[[118, 129, 931, 910, 8, 669, 9, 30, 28, 1225]]

In [56]:
result_X = sequence.pad_sequences(result_data, maxlen=max_sentence_len)
# result_data.extend([0] * (59 - len(result_data)))

In [57]:
result_X

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,  118,  129,  931,  910,    8,  669,
           9,   30,   28, 1225]], dtype=int32)

In [58]:
result_y_pred = loadedModel.predict(result_X, batch_size=batch_size)

In [59]:
result_y_pred.ravel()

array([0.00025697], dtype=float32)